In [26]:
import numpy as np
import argparse
import os
import glob

import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import Callback

import tensorflow as tf

from azureml.core import Run
from utils import load_data, one_hot_encode
from sklearn.metrics import precision_score

In [1]:
from azureml.core.run import Run
from azureml.core import Dataset, Datastore, Workspace
import os
import argparse
import joblib
import json
from keras_mnist import preprocess_df, train_model, get_model_metrics

In [3]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: mlops-aml-ws
Azure region: centralus
Subscription id: 67c8a1c5-3a57-45c1-afa7-2e780c1372da
Resource group: mlops-rg


In [13]:
web_paths = ['https://azureopendatastorage.blob.core.windows.net/mnist/train-images-idx3-ubyte.gz',
             'https://azureopendatastorage.blob.core.windows.net/mnist/train-labels-idx1-ubyte.gz',
             'https://azureopendatastorage.blob.core.windows.net/mnist/t10k-images-idx3-ubyte.gz',
             'https://azureopendatastorage.blob.core.windows.net/mnist/t10k-labels-idx1-ubyte.gz'
            ]
dataset = Dataset.File.from_files(path = web_paths)

In [21]:
dataset = dataset.register(workspace = ws,
                            name = 'mnist-dataset',
                            description='training and test dataset',
                            create_new_version=True)

In [23]:
Dataset.get_by_name(ws,"mnist-dataset")

{
  "source": [
    "https://azureopendatastorage.blob.core.windows.net/mnist/train-images-idx3-ubyte.gz",
    "https://azureopendatastorage.blob.core.windows.net/mnist/train-labels-idx1-ubyte.gz",
    "https://azureopendatastorage.blob.core.windows.net/mnist/t10k-images-idx3-ubyte.gz",
    "https://azureopendatastorage.blob.core.windows.net/mnist/t10k-labels-idx1-ubyte.gz"
  ],
  "definition": [
    "GetFiles"
  ],
  "registration": {
    "id": "98c44951-5d10-489a-a7d4-3162d322cfb9",
    "name": "mnist-dataset",
    "version": 1,
    "description": "training and test dataset",
    "workspace": "Workspace.create(name='mlops-aml-ws', subscription_id='67c8a1c5-3a57-45c1-afa7-2e780c1372da', resource_group='mlops-rg')"
  }
}

In [17]:
dataset

{
  "source": [
    "https://azureopendatastorage.blob.core.windows.net/mnist/train-images-idx3-ubyte.gz",
    "https://azureopendatastorage.blob.core.windows.net/mnist/train-labels-idx1-ubyte.gz",
    "https://azureopendatastorage.blob.core.windows.net/mnist/t10k-images-idx3-ubyte.gz",
    "https://azureopendatastorage.blob.core.windows.net/mnist/t10k-labels-idx1-ubyte.gz"
  ],
  "definition": [
    "GetFiles"
  ],
  "registration": {
    "id": "98c44951-5d10-489a-a7d4-3162d322cfb9",
    "name": "mnist-dataset",
    "version": 1,
    "description": "training and test dataset",
    "workspace": "Workspace.create(name='mlops-aml-ws', subscription_id='67c8a1c5-3a57-45c1-afa7-2e780c1372da', resource_group='mlops-rg')"
  }
}

In [12]:
model_name = args.model_name
step_output_path = args.step_output
dataset_version = args.dataset_version
data_file_path = args.data_file_path
dataset_name = args.dataset_name

In [15]:
# Load the training parameters from the parameters file
with open("parameters.json") as f:
    pars = json.load(f)
try:
    train_args = pars["training"]
except KeyError:
    print("Could not load training values from file")
    train_args = {}

In [16]:
train_args

{'n_inputs': 784,
 'n_h1': 300,
 'n_h2': 100,
 'n_outputs': 10,
 'n_epochs': 20,
 'batch_size': 50,
 'learning_rate': 0.001}

In [18]:
train_args["n_inputs"]
train_args["n_inputs"]
train_args["n_inputs"]
train_args["n_inputs"]
train_args["n_inputs"]
train_args["n_inputs"]
train_args["n_inputs"]

784

In [39]:
data_folder = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/temp/code/Users/lydia.ingabire.1/mnist/data'

In [43]:
glob.glob(os.path.join(data_folder, '**/train-images-idx3-ubyte.gz'), recursive=True)[0]

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/temp/code/Users/lydia.ingabire.1/mnist/data/train-images-idx3-ubyte.gz'

In [44]:
datastore = Datastore.get(aml_workspace, datastore_name)
dataset = Dataset.Tabular.from_delimited_files(path=(datastore, file_path))
dataset = dataset.register(workspace=aml_workspace,
                               name=dataset_name,
                               create_new_version=True)

NameError: name 'Datastore' is not defined